In [ ]:
import torch
import torch.nn as nn
import random
import time
import math
import csv 
import string
import unicodedata

In [ ]:
all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 

def unicodeToAscii(s):
    """Перетворює Unicode рядок на простий ASCII."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def load_character_data_from_csv(filepath):
    """
    Завантажує дані з CSV, створює словник
    де ключ - це media_source (категорія),
    а значення - список імен персонажів.
    """
    category_lines = {}
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            char_name = unicodeToAscii(row['character_name'])
            media_source = row['media_source']
            
            if not char_name: 
                continue
                
            if media_source not in category_lines:
                category_lines[media_source] = []
            category_lines[media_source].append(char_name)
    
    all_categories = list(category_lines.keys())
    return category_lines, all_categories

category_lines, all_categories = load_character_data_from_csv('data.csv')
n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Дані не знайдено або файл data.csv порожній.')

print(f'# категорій знайдено: {n_categories}')
print('Приклад категорій:', all_categories[:10]) 

# --- Створення мережі ---

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# --- Підготовка до навчання ---

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

# --- Навчання мережі  ---

criterion = nn.NLLLoss()
learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()
    rnn.zero_grad()
    loss = 0
    for i in range(input_line_tensor.size()[0]):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l
    loss.backward()
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
    return output, loss.item() / input_line_tensor.size()[0]

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# --- Запуск процесу навчання ---

rnn = RNN(n_letters, 128, n_letters)
n_iters = 100000 
print_every = 5000
start = time.time()

print("\nПочинаємо навчання...")
for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

print("Навчання завершено.")

# --- Семплінг (Генерація)  ---

max_length = 25 

def sample(category, start_letter='A'):
    with torch.no_grad():
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()
        output_name = start_letter
        for i in range(max_length - 1):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)
        return output_name

def samples(category, start_letters='ABC'):
    print(f'\n--- Генерація імен для: {category} ---')
    for start_letter in start_letters:
        print(sample(category, start_letter))

# --- Генерація імен персонажів ---


samples('The Lord of the Rings', 'SFG')
samples('Star Wars', 'LPO')
samples('Harry Potter', 'RHD')
samples('Marvel Cinematic Universe', 'TSB')

# категорій знайдено: 9061
Приклад категорій: ['Saga', 'The Smiling, Proud Wanderer', 'Atomic Robo', 'Fairy Tail', "Maus: A Survivor's Tale", 'Uncertainty Principle', 'The Adventures of Tintin', 'Historical Scientist', 'Marvel Comics', 'Demon Slayer (Kimetsu no Yaiba)']

Починаємо навчання...
0m 57s (5000 5%) 3.0581
1m 58s (10000 10%) 3.0132
2m 56s (15000 15%) 3.2252
3m 55s (20000 20%) 2.6886
4m 55s (25000 25%) 3.1093
5m 54s (30000 30%) 2.3778
6m 52s (35000 35%) 2.7192
7m 49s (40000 40%) 3.1114
8m 46s (45000 45%) 2.6076
9m 44s (50000 50%) 3.2395
10m 45s (55000 55%) 2.8165
11m 41s (60000 60%) 2.3940
12m 39s (65000 65%) 2.1980
13m 41s (70000 70%) 3.2014
14m 41s (75000 75%) 3.2001
15m 44s (80000 80%) 2.2169
16m 42s (85000 85%) 2.6527
17m 38s (90000 90%) 2.2308
18m 34s (95000 95%) 2.6930
19m 31s (100000 100%) 2.3293
Навчання завершено.

--- Генерація імен для: The Lord of the Rings ---
Santer Santer
Frand Santer
Grin Santer

--- Генерація імен для: Star Wars ---
Lane Sarane Sarde
Peris San